In [ ]:
# DO NOT REMOVE

# Compute double truncated GR from seismicity

In [11]:
%matplotlib inline
import os
import re
import sys
import h5py
import numpy
try:
    import cPickle as pickle
except:
    import pickle

from collections import Counter
from prettytable import PrettyTable

from openquake.hmtk.seismicity.occurrence.weichert import Weichert
from openquake.hmtk.seismicity.occurrence.utils import get_completeness_counts
from openquake.hmtk.plotting.seismicity.occurrence.recurrence_plot import plot_trunc_gr_model

from oqmbt.oqt_project import OQtProject
from oqmbt.tools.area import create_catalogue

In [12]:
project_pickle_filename = os.environ.get('OQMBT_PROJECT')
oqtkp = OQtProject.load_from_file(project_pickle_filename)
print ('Project filename:', project_pickle_filename)
model_id = oqtkp.active_model_id
model = oqtkp.models[model_id]
#
# hdf5 files
compl_hdf5_filename = os.path.join(oqtkp.directory, oqtkp.compl_hdf5_filename)
eqk_rates_hdf5_filename = os.path.join(oqtkp.directory, oqtkp.eqk_rates_hdf5_filename)
#
# set source ID
try:
    area_source_ids_list = getattr(oqtkp,'active_source_id')
except:
    print ('Active source ID not defined in the OQMBT project')
    area_source_ids_list = ['1']
#
# info 
print ('Processing model with ID:', model_id)
print ('Processing area source with ID:', area_source_ids_list)

Project filename: /Users/kjohnson/GEM/Regions/paisl18/project/paisl.oqmbtp
Processing model with ID: model01
Processing area source with ID: ['1']


In [ ]:
# Width of bins used to discretise the MFDs
binwidth = float(model.mfd_binwidth)

## Summary of GR parameters assigned to area sources

In [ ]:
# Set table
p = PrettyTable(["ID","a_gr", "b_gr"])
p.align["Source ID"] = 'l'
p.align["a_gr"] = 'r'
p.align["b_gr"] = 'r'
#
for key in sorted(model.sources):
    src = model.sources[key]
    if src.source_type == 'AreaSource':
        alab = ''
        blab = ''
        if 'a_gr' in src.__dict__:
            alab = '%8.5f' % (src.a_gr)
        if 'b_gr' in src.__dict__:
            blab = '%6.3f' % (src.b_gr)    
        p.add_row([key, alab, blab])
print (p)

## Read catalogue

In [ ]:
print (oqtkp.models[model_id].declustered_catalogue_pickle_filename)
pickle_filename = os.path.join(oqtkp.directory, oqtkp.models[model_id].declustered_catalogue_pickle_filename)
fin = open(pickle_filename, 'rb') 
catalogue = pickle.load(fin)
fin.close()
print ('The catalogue contains %d earthquakes' % (len(catalogue.data['magnitude'])))

### Create catalogue for the selected areas

In [ ]:
fcatal = create_catalogue(model, catalogue, area_source_ids_list)

In [ ]:
mmin_model = float(model.catalogue_cutoff_magnitude)
print ('Minimum magnitude considered for the model is: ', mmin_model)
mmax_obs = float(max(fcatal.data['magnitude']))
print ('Maximum observed magnitude is: ', mmax_obs)

### Load the completeness table

In [ ]:
compl_hdf5_filename = os.path.join(oqtkp.directory, oqtkp.compl_hdf5_filename)
print ('Reading {:s}\n'.format(compl_hdf5_filename))
fhdf5 = h5py.File(compl_hdf5_filename,'r')

grp = fhdf5[model_id]
src_id = area_source_ids_list[0]
if src_id in grp.keys():
    compl_table = grp[src_id][()]
    print ('Found completeness table for: <%s>' % (src_id))
else:
    print ('The .hdf5 file does not contain completeness table for source {:s}'.format(src_id))
    if 'whole_catalogue' in grp.keys():
        compl_table = grp['whole_catalogue'][()]
        print ('Using the default completeness table set for the whole catalogue')
    else:
        print ('Default completeness table (whole catalogue) not defined')
        raise ValueError()
        compl_table = None
print ('\nCompleteness table:')
print (compl_table)
fhdf5.close()

## Double truncated magnitude-frequency distribution (MFD)

In [ ]:
#
# selecting earthquakes
idx = numpy.nonzero(compl_table[:,1] < numpy.max(fcatal.data['magnitude']))
weichert_config = {'magnitude_interval': 0.1, 
                   'reference_magnitude': 0.0}
weichert = Weichert()
bval_wei, sigmab, aval_wei, sigmaa = weichert.calculate(fcatal, weichert_config, 
                                                        completeness=compl_table)
#
# info
print ('bval: %.6f (sigma=%.3f)' % (bval_wei, sigmab))
print ('aval: %.6f (sigma=%.3f)' % (aval_wei, sigmaa))
#
# computing seismicity rates
cent_mag, t_per, n_obs = get_completeness_counts(fcatal, compl_table, binwidth)

In [ ]:
print('Updating', eqk_rates_hdf5_filename)
fhdf5 = h5py.File(eqk_rates_hdf5_filename, 'a')
#
# update/create group
if model_id in fhdf5.keys():
    print ('    Group exists. Set group %s' % (model_id))
    grp = fhdf5[model_id]
else:
    print ('    Create group: %s' % (model_id))
    grp = fhdf5.create_group(model_id)
#
# Update/create dataset
rates = numpy.array([cent_mag, t_per, n_obs])
if src_id in grp:
    del grp[src_id]
print('    Creating dataset %s' % (src_id))
dataset = grp.create_dataset(src_id, data=rates)
fhdf5.close()

In [ ]:
plot_trunc_gr_model(aval_wei, bval_wei, mmin_model, mmax_obs, binwidth, catalogue=fcatal,
        completeness=compl_table)

In [ ]:
#
# Update the project info
c = Counter(fcatal.data['comment'])
weights = {}
annual_rate_source = []
#
# 
for srcid in area_source_ids_list:
    #
    # Computing weight
    num_eqs_source = float(c['%s' % srcid])
    weights[srcid] = num_eqs_source / fcatal.get_number_events()
    print ('source', srcid, \
           ' | num. of eqks [eqks] %6.2f' % (c['%s' % srcid]), \
           ' | weight %s' % (weights[srcid]))
    #
    # attaching source to model 
    src = model.get_source(srcid) 
    src.a_gr=numpy.log10(10**aval_wei * weights[srcid])
    src.b_gr=bval_wei
#
# saving the project
oqtkp.save(log=True)
#
# check of the weight
checkw = 0.0
for key in weights:
    checkw += weights[key]
print ('check:', checkw)

In [ ]:
# Set table
p = PrettyTable(["ID","a_gr", "b_gr"])
p.align["Source ID"] = 'l'
p.align["a_gr"] = 'r'
p.align["b_gr"] = 'r'
#
for key in sorted(model.sources):
    src = model.sources[key]
    if src.source_type == 'AreaSource':
        alab = ''
        blab = ''
        if 'a_gr' in src.__dict__:
            alab = '%8.5f' % (src.a_gr)
        if 'b_gr' in src.__dict__:
            blab = '%6.3f' % (src.b_gr)    
        p.add_row([key, alab, blab])
print (p)